In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from keras.models import Model
from keras.layers import Dense, Input, add, BatchNormalization
from tensorflow.keras.optimizers import Nadam
from keras.layers.core import Dropout, Activation
from keras.initializers import RandomNormal, Ones

import seaborn as sns

import random
from numpy import array
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
def predict_dist(X, model, num_samples):
    preds = [model(X, training=True) for _ in range(num_samples)]
    return np.hstack(preds)

def predict_point(X, model, num_samples):
    pred_dist = predict_dist(X, model, num_samples)
    return pred_dist.mean(axis=1), pred_dist

In [ ]:
# Function PD: Calculate Poisson Deviance
def PD_function(pred, obs):
    PD = 200*( sum(pred) - sum(obs) + sum( np.log( (obs/pred)**(obs) ) ) )
    return PD/len(pred)


# evaluate a single mlp model
def evaluate_model(trainX, trainy, train_expo):

    first_input = Input(shape=(14, ))

    first_dense_00 = Dense(50)(first_input)
    act_dense_00 = Activation("sigmoid")(first_dense_00)
    act_dense_00 = BatchNormalization()(act_dense_00)
    Dropout_dense_00 = Dropout(0.3)(act_dense_00)

    first_dense_0 = Dense(25)(Dropout_dense_00)
    act_dense_0 = Activation("sigmoid")(first_dense_0)
    act_dense_0 = BatchNormalization()(act_dense_0)
    Dropout_dense_0 = Dropout(0.2)(act_dense_0)

    first_dense_1 = Dense(15)(Dropout_dense_0)
    act_dense_1 = Activation("sigmoid")(first_dense_1)
    act_dense_1 = BatchNormalization()(act_dense_1)
    Dropout_dense_1 = Dropout(0.1)(act_dense_1)

    first_dense_2 = Dense(8)(Dropout_dense_1)
    act_dense_2 = Activation("sigmoid")(first_dense_2)
    act_dense_2 = BatchNormalization()(act_dense_2)
    Dropout_dense_2 = Dropout(0.1)(act_dense_2)

    first_dense_3 = Dense(1)(Dropout_dense_2)
    act_dense_3 = Activation('linear')(first_dense_3)

    second_input = Input(shape=(1, ))

    second_dense_1 = Dense(1)(add([act_dense_3, second_input])) 
    merge_one = Activation('exponential')(second_dense_1) 

    model_tanh = Model(inputs=[first_input, second_input], outputs=merge_one)

    model_tanh.compile(optimizer=Nadam(), loss='poisson')
    
    model_tanh.fit([trainX, train_expo], trainy, epochs=450, verbose=0,
                   batch_size=10000
                  )

    return model_tanh

, 81, 37 ,71, 28, 70, 54, 59, 17, 79

, 54, 42, 88, 29

In [ ]:
df_poisson_10 = pd.DataFrame()
df_poisson_20 = pd.DataFrame()
df_poisson_30 = pd.DataFrame()
df_poisson_40 = pd.DataFrame()

table_name = ["deviance_train", "deviance_test", "diff_train", "diff_test"]


for p in [62, 54, 42, 88, 29]:
    
    print(p)
    
    data_learn = pd.read_csv(r'C:\Users\KRASNIQ\Documents\These - Copie\data\data_learn_'+str(62)+'.csv')
    data_test = pd.read_csv(r'C:\Users\KRASNIQ\Documents\These - Copie\data\data_test_'+str(62)+'.csv')
    
    nex_data = pd.concat([data_learn, data_test])
    
    data_learn_, data_test_ = train_test_split(nex_data, stratify=nex_data['ClaimNb'], test_size=0.1, random_state=p)
    
    data_learn_['Exposure'] = data_learn_['Exposure'].clip(upper=1)
    data_test_['Exposure'] = data_test_['Exposure'].clip(upper=1)
    
    data_learn_['Area'] = data_learn_['Area'].map({"A": 1, "B": 2,"C": 3,"D": 4,"E": 5,"F": 6,})
    data_test_['Area'] = data_test_['Area'].map({"A": 1, "B": 2,"C": 3,"D": 4,"E": 5,"F": 6,})
    
    data_learn_ = data_learn_.drop(["Area_class_1","Area_class_2","Area_class_3","Area_class_4",'ClaimAmount', 'DrivAge', 'BonusMalus','VehBrand',
                                       'Density', 'Region', 'VehAge', 'VehGas', "VehGas_class_2"], axis=1)

    data_test_ = data_test_.drop(["Area_class_1","Area_class_2","Area_class_3","Area_class_4",'ClaimAmount', 'DrivAge', 'BonusMalus','VehBrand',
                                    'Density', 'Region', 'VehAge', 'VehGas', "VehGas_class_2"], axis=1)
    
    # train test split 
    X_train__, y_valid__ = train_test_split(data_learn_, stratify=data_learn_['ClaimNb'], test_size=0.1, random_state=p)
    
    y_train = np.array(X_train__.filter(['ClaimNb']))
    log_exp_train= np.array(np.log(X_train__.filter(['Exposure'])))
    x_train = X_train__.drop(columns=['ClaimNb', 'Exposure'])

    y_valid = np.array(y_valid__.filter(['ClaimNb']))
    log_exp_valid= np.array(np.log(y_valid__.filter(['Exposure'])))
    x_valid = y_valid__.drop(columns=['ClaimNb', 'Exposure'])
    
    #valid
    y_test = np.array(data_test_.filter(['ClaimNb']))
    log_exp_test = np.array(np.log(data_test_.filter(['Exposure'])))
    x_test = data_test_.drop(columns=['ClaimNb', 'Exposure'])
    
    #feature eng
    cs = MinMaxScaler()
    X_train = cs.fit_transform(x_train)
    X_valid = cs.fit_transform(x_valid)
    X_test = cs.transform(x_test)
    
    print(X_train.shape)
    model = evaluate_model(X_train, y_train, log_exp_train)
    
    
    ############################### strategie 1 ##########################################################
    
    print('strategie Drop Out')
    


    print('10')
    predictions_test_10, pred_dist_test_10 = predict_point([X_test, log_exp_test], model, 10)
    predictions_train_10, pred_dist_train_10 = predict_point([X_train, log_exp_train], model, 10)

    #deviance
    table_modal= [np.round(PD_function(predictions_train_10, y_train.flatten()),6),
                  np.round(PD_function(predictions_test_10, y_test.flatten()),6),
                  (sum(predictions_train_10) - sum(y_train.flatten()))/sum(y_train.flatten())*100,
                  (sum(predictions_test_10) -sum(y_test.flatten()))/sum(y_test.flatten())*100]

    cnt = dict()
    k=0
    for word in table_name:
        cnt[word] = table_modal[k]
        k = k+1
    df_poisson_10 = df_poisson_10.append(cnt, ignore_index=True)
    
    
    

    print('20')    
    predictions_test_20, pred_dist_test_20 = predict_point([X_test, log_exp_test], model, 20)
    predictions_train_20, pred_dist_train_20 = predict_point([X_train, log_exp_train], model, 20)

    #deviance
    table_modal= [np.round(PD_function(predictions_train_20, y_train.flatten()),6),
                  np.round(PD_function(predictions_test_20, y_test.flatten()),6),
                  (sum(predictions_train_20) - sum(y_train.flatten()))/sum(y_train.flatten())*100,
                  (sum(predictions_test_20) -sum(y_test.flatten()))/sum(y_test.flatten())*100]

    cnt = dict()
    k=0
    for word in table_name:
        cnt[word] = table_modal[k]
        k = k+1
    df_poisson_20 = df_poisson_20.append(cnt, ignore_index=True)
    
    
    
    

    print('30')
    predictions_test_30, pred_dist_test_30 = predict_point([X_test, log_exp_test], model, 50)
    predictions_train_30, pred_dist_train_30 = predict_point([X_train, log_exp_train], model, 50)

    #deviance
    table_modal= [np.round(PD_function(predictions_train_30, y_train.flatten()),6),
                  np.round(PD_function(predictions_test_30, y_test.flatten()),6),
                  (sum(predictions_train_30) - sum(y_train.flatten()))/sum(y_train.flatten())*100,
                  (sum(predictions_test_30) -sum(y_test.flatten()))/sum(y_test.flatten())*100]

    cnt = dict()
    k=0
    for word in table_name:
        cnt[word] = table_modal[k]
        k = k+1
    df_poisson_30 = df_poisson_30.append(cnt, ignore_index=True)

    
    
    print('40')
    predictions_test_40, pred_dist_test_40 = predict_point([X_test, log_exp_test], model, 50)
    predictions_train_40, pred_dist_train_40 = predict_point([X_train, log_exp_train], model, 50)

    #deviance
    table_modal= [np.round(PD_function(predictions_train_40, y_train.flatten()),6),
                  np.round(PD_function(predictions_test_40, y_test.flatten()),6),
                  (sum(predictions_train_40) - sum(y_train.flatten()))/sum(y_train.flatten())*100,
                  (sum(predictions_test_40) -sum(y_test.flatten()))/sum(y_test.flatten())*100]

    cnt = dict()
    k=0
    for word in table_name:
        cnt[word] = table_modal[k]
        k = k+1
    df_poisson_40 = df_poisson_40.append(cnt, ignore_index=True)

In [ ]:
df_poisson_10

In [ ]:
df_poisson_10["deviance_train"].mean(), df_poisson_10["deviance_test"].mean()

In [ ]:
df_poisson_10["deviance_train"].std(), df_poisson_10["deviance_test"].std()

In [ ]:
print("ensemble", predictions_test_10[35])
print("target", y_test[35])
sns.kdeplot(pred_dist_test_10[35], shade=True)
plt.axvline(predictions_test_10[35], color='red')
plt.show()

In [ ]:
df_poisson_20

In [ ]:
df_poisson_20["deviance_train"].mean(), df_poisson_20["deviance_test"].mean()

In [ ]:
(30.117755999999996, 30.144133600000004)

In [ ]:
df_poisson_20["deviance_train"].std(), df_poisson_20["deviance_test"].std()

In [ ]:
(30.117755999999996, 30.144133600000004)

In [ ]:
print("ensemble", predictions_test_20[35])
print("target", y_test[35])
sns.kdeplot(pred_dist_test_20[35], shade=True)
plt.axvline(predictions_test_20[35], color='red')
plt.show()

In [ ]:
df_poisson_30

In [ ]:
df_poisson_30["deviance_train"].mean(), df_poisson_30["deviance_test"].mean()

In [ ]:
df_poisson_30["deviance_train"].std(), df_poisson_30["deviance_test"].std()

In [ ]:
print("ensemble", predictions_test_30[35])
print("target", y_test[35])
sns.kdeplot(pred_dist_test_30[35], shade=True)
plt.axvline(predictions_test_30[35], color='red')
plt.show()

In [ ]:
df_poisson_40

In [ ]:
df_poisson_40["deviance_train"].mean(), df_poisson_40["deviance_test"].mean()

In [ ]:
df_poisson_40["deviance_train"].std(), df_poisson_40["deviance_test"].std()

In [ ]:
print("ensemble", predictions_test_40[35])
print("target", y_test[35])
sns.kdeplot(pred_dist_test_40[35], shade=True)
plt.axvline(predictions_test_40[35], color='red')
plt.show()